<a href="https://colab.research.google.com/github/tronglinux123/AI-Project/blob/main/cohortselection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datetime import timedelta
import os

import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from IPython.display import display, HTML, Image
%matplotlib inline

plt.style.use('ggplot')
plt.rcParams.update({'font.size': 20})

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

In [ ]:
# Set up environment variables
project_id = 'project-physionet-474410'
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id

# Read data from BigQuery into pandas dataframes.
def run_query(query, project_id=project_id):
  return pd.io.gbq.read_gbq(
      query,
      project_id=project_id,
      dialect='standard')

# set the dataset
dataset = 'mimiciv'


In [ ]:
df = run_query("""
SELECT icd_code, long_title FROM `physionet-data.mimiciv_2_2_hosp.d_icd_diagnoses`
WHERE LOWER(long_title) LIKE "%pneumonia%" OR LOWER(long_title) LIKE "%tuberculosis%" OR LOWER(long_title) LIKE "%bronchitis%"
""")

# ICD code list
icd_codes = df['icd_code'].to_list()
icd_codes_string = ', '.join(["'" + icd + "'" for icd in icd_codes])
# Subject ID that have an ICD code related to pneumonia, tuberculosis and bronchitis and a CHEST x-ray

all_subject_ids = run_query(f"""
SELECT `physionet-data.mimiciv_2_2_hosp.patients`.subject_id
FROM `physionet-data.mimiciv_2_2_hosp.patients`
JOIN `physionet-data.mimiciv_2_2_hosp.admissions` ON `physionet-data.mimiciv_2_2_hosp.patients`.subject_id = `physionet-data.mimiciv_2_2_hosp.admissions`.subject_id
JOIN `physionet-data.mimiciv_2_2_hosp.diagnoses_icd` ON `physionet-data.mimiciv_2_2_hosp.admissions`.hadm_id = `physionet-data.mimiciv_2_2_hosp.diagnoses_icd`.hadm_id
JOIN `physionet-data.mimic_cxr.record_list` ON `physionet-data.mimiciv_2_2_hosp.patients`.subject_id = `physionet-data.mimic_cxr.record_list`.subject_id
JOIN `physionet-data.mimic_cxr.dicom_metadata_string` ON `physionet-data.mimic_cxr.record_list`.dicom_id = `physionet-data.mimic_cxr.dicom_metadata_string`.dicom
WHERE `physionet-data.mimiciv_2_2_hosp.diagnoses_icd`.icd_code IN ({icd_codes_string}) AND `physionet-data.mimic_cxr.dicom_metadata_string`.BodyPartExamined = 'CHEST'
GROUP BY `physionet-data.mimiciv_2_2_hosp.patients`.subject_id
""")
print(all_subject_ids)


/tmp/ipython-input-2544362523.py:7: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


       subject_id
0        18415616
1        19136768
2        16255488
3        10554112
4        13046528
...           ...
11329    11917055
11330    17686783
11331    10150911
11332    16513279
11333    15007487

[11334 rows x 1 columns]


In [ ]:
subject_hadm = run_query(f"""
SELECT
    `physionet-data.mimiciv_2_2_hosp.patients`.subject_id,
    ARRAY_AGG(DISTINCT `physionet-data.mimiciv_2_2_hosp.admissions`.hadm_id) AS hadm_ids
FROM
    `physionet-data.mimiciv_2_2_hosp.patients`
JOIN
    `physionet-data.mimiciv_2_2_hosp.admissions` ON `physionet-data.mimiciv_2_2_hosp.patients`.subject_id = `physionet-data.mimiciv_2_2_hosp.admissions`.subject_id
JOIN
    `physionet-data.mimiciv_2_2_hosp.diagnoses_icd` ON `physionet-data.mimiciv_2_2_hosp.admissions`.hadm_id = `physionet-data.mimiciv_2_2_hosp.diagnoses_icd`.hadm_id
JOIN
    `physionet-data.mimic_cxr.record_list` ON `physionet-data.mimiciv_2_2_hosp.patients`.subject_id = `physionet-data.mimic_cxr.record_list`.subject_id
JOIN
    `physionet-data.mimic_cxr.dicom_metadata_string` ON `physionet-data.mimic_cxr.record_list`.dicom_id = `physionet-data.mimic_cxr.dicom_metadata_string`.dicom
WHERE
    `physionet-data.mimiciv_2_2_hosp.diagnoses_icd`.icd_code IN ({icd_codes_string})
AND
    `physionet-data.mimic_cxr.dicom_metadata_string`.BodyPartExamined = 'CHEST'
GROUP BY
    `physionet-data.mimiciv_2_2_hosp.patients`.subject_id

""")
subject_hadm

/tmp/ipython-input-2544362523.py:7: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


,subject_id,hadm_ids
0,10361837,"[23346424, 22076746, 27515984]"
1,10433099,[28593361]
2,12481952,"[24985536, 27017213, 27857372, 22058685, 20146..."
3,12862321,"[27343985, 22439772, 26167368]"
4,16662264,"[22174937, 26390489, 21137571, 21588032, 28552..."
...,...,...
11329,18505859,[20886432]
11330,10940509,[26533493]
11331,11080025,[25064210]
11332,13588348,[29652751]


In [ ]:
subject_hadm_icd = run_query(f"""
SELECT
    `physionet-data.mimiciv_2_2_hosp.patients`.subject_id,
    `physionet-data.mimiciv_2_2_hosp.admissions`.hadm_id,
    ARRAY_AGG(DISTINCT `physionet-data.mimiciv_2_2_hosp.diagnoses_icd`.icd_code) AS icd_codes
FROM
    `physionet-data.mimiciv_2_2_hosp.patients`
JOIN
    `physionet-data.mimiciv_2_2_hosp.admissions` ON `physionet-data.mimiciv_2_2_hosp.patients`.subject_id = `physionet-data.mimiciv_2_2_hosp.admissions`.subject_id
JOIN
    `physionet-data.mimiciv_2_2_hosp.diagnoses_icd` ON `physionet-data.mimiciv_2_2_hosp.admissions`.hadm_id = `physionet-data.mimiciv_2_2_hosp.diagnoses_icd`.hadm_id
JOIN
    `physionet-data.mimic_cxr.record_list` ON `physionet-data.mimiciv_2_2_hosp.patients`.subject_id = `physionet-data.mimic_cxr.record_list`.subject_id
JOIN
    `physionet-data.mimic_cxr.dicom_metadata_string` ON `physionet-data.mimic_cxr.record_list`.dicom_id = `physionet-data.mimic_cxr.dicom_metadata_string`.dicom
WHERE
    `physionet-data.mimiciv_2_2_hosp.diagnoses_icd`.icd_code IN ({icd_codes_string})
AND
    `physionet-data.mimic_cxr.dicom_metadata_string`.BodyPartExamined = 'CHEST'
GROUP BY
    `physionet-data.mimiciv_2_2_hosp.patients`.subject_id, `physionet-data.mimiciv_2_2_hosp.admissions`.hadm_id
""")
subject_hadm_icd

/tmp/ipython-input-2544362523.py:7: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


,subject_id,hadm_id,icd_codes
0,10361837,23346424,[4829]
1,10433099,28593361,"[48283, 99731]"
2,12481952,24985536,[4870]
3,12862321,27343985,[4829]
4,16662264,22174937,[486]
...,...,...,...
18631,15416020,25055834,[J189]
18632,16488736,26871531,[J189]
18633,18026603,22204850,[B961]
18634,18136989,27878914,[J189]


In [ ]:
subject_hadm_icd_dicom = run_query(f"""
WITH unique_dicoms AS (
    SELECT DISTINCT
        rl.subject_id,
        rl.dicom_id
    FROM
        `physionet-data.mimic_cxr.record_list` rl
    JOIN
        `physionet-data.mimic_cxr.dicom_metadata_string` dms ON rl.dicom_id = dms.dicom
    WHERE
        dms.BodyPartExamined = 'CHEST'
    AND
        dms.ViewPosition IN ('AP', 'PA', 'LATERAL', 'LL')
)

SELECT
    p.subject_id,
    a.hadm_id,
    ARRAY_AGG(DISTINCT di.icd_code) AS icd_codes,
    ARRAY_AGG(DISTINCT ud.dicom_id) AS dicom_ids
FROM
    `physionet-data.mimiciv_2_2_hosp.patients` p
JOIN
    `physionet-data.mimiciv_2_2_hosp.admissions` a ON p.subject_id = a.subject_id
JOIN
    `physionet-data.mimiciv_2_2_hosp.diagnoses_icd` di ON a.hadm_id = di.hadm_id
JOIN
    unique_dicoms ud ON p.subject_id = ud.subject_id
WHERE
    di.icd_code IN ({icd_codes_string})
GROUP BY
    p.subject_id, a.hadm_id

""")
subject_hadm_icd_dicom

/tmp/ipython-input-2544362523.py:7: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


,subject_id,hadm_id,icd_codes,dicom_ids
0,10361837,23346424,[4829],"[5ba8a2a0-25dfdb77-186f5527-86f8ebbf-bc15cf2c,..."
1,10433099,28593361,"[48283, 99731]","[8ec64b03-7dea1acb-52f75412-958dd493-397e43d5,..."
2,12481952,24985536,[4870],"[9229b1fc-97966f5c-ff3d4022-b2bcff57-08140b76,..."
3,12862321,27343985,[4829],"[d475bdab-0547132f-e52305a1-04351d27-bce59c75,..."
4,16662264,22174937,[486],"[0211e30b-241ba3f8-9258eace-8ee95a4b-ecac4b96,..."
...,...,...,...,...
18481,15416020,25055834,[J189],"[42761bab-bc16f8ed-7af3162c-d140953e-9c0f468c,..."
18482,16488736,26871531,[J189],"[d97fb13a-ead0f345-af088862-9d3e2464-9232e965,..."
18483,18026603,22204850,[B961],"[24a2d6f7-a15d2843-bcad811f-09b35037-48d51125,..."
18484,18136989,27878914,[J189],"[72a8dd12-d9dc6bb9-6f36b80c-5ed3bf81-a98af4e3,..."


In [ ]:
dicom_mono = run_query(f"""
SELECT
    dicom,
    PhotometricInterpretation
FROM
    `physionet-data.mimic_cxr.dicom_metadata_string`
WHERE
    PhotometricInterpretation IN ('MONOCHROME1', 'MONOCHROME2')
""")
dicom_mono

/tmp/ipython-input-2544362523.py:7: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


,dicom,PhotometricInterpretation
0,84219406-37f6345f-a1a2dff6-6702b8fe-2c306bf2,MONOCHROME2
1,3a0ef154-76c7771c-c20547c1-e458ee9c-9d9c25e6,MONOCHROME2
2,62c96ddd-238381c1-80a7b2e8-dddef99a-d29aa6c2,MONOCHROME2
3,acbfadb0-75d7c999-d5c5b73b-bb53fce3-9a11899e,MONOCHROME2
4,43165744-14cbd7a0-a0c1ec94-c7b57dd1-4c7454a6,MONOCHROME2
...,...,...
377105,67fbb595-ac47306d-90c1b2d0-3b790fde-c496459f,MONOCHROME2
377106,9c9db31d-cd450363-520410ce-c1c3f5dd-7c79bbf8,MONOCHROME2
377107,a43142f0-504e9beb-f5710f72-fb264e8b-1a8d6b9c,MONOCHROME2
377108,6cc273a6-c4a2fb7b-572d6f4f-4b5caf6a-9e1fc1eb,MONOCHROME2


In [ ]:
# Expand the 'dicom_ids' column to multiple rows
s = subject_hadm_icd_dicom.apply(lambda x: pd.Series(x['dicom_ids']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'dicom'
expanded_df = subject_hadm_icd_dicom.drop('dicom_ids', axis=1).join(s)

# Merge with the dicom_mono dataframe
merged_df = pd.merge(expanded_df, dicom_mono, on='dicom', how='left')
merged_df


,subject_id,hadm_id,icd_codes,dicom,PhotometricInterpretation
0,10361837,23346424,[4829],5ba8a2a0-25dfdb77-186f5527-86f8ebbf-bc15cf2c,MONOCHROME2
1,10361837,23346424,[4829],f7edd035-257a964c-3faaede6-ec9f82b1-4b627905,MONOCHROME2
2,10361837,23346424,[4829],5652d00b-66cda672-eb339193-48106ee3-687c7590,MONOCHROME2
3,10361837,23346424,[4829],47645fa6-adcb2a36-f77b72a8-9f71497e-b19fbb45,MONOCHROME2
4,10361837,23346424,[4829],e30ad024-9ebe3818-4291d1a7-e6b6c20c-4c87b23e,MONOCHROME2
...,...,...,...,...,...
297117,15132645,22796487,[R7611],a60476a5-1be933f6-5db97c2e-81ccab31-7f43afe9,MONOCHROME2
297118,15132645,22796487,[R7611],a6f5f1a5-d0e8bebf-3367688c-f38c00cf-3af18863,MONOCHROME2
297119,15132645,22796487,[R7611],1f17c382-3bb7aaff-009fdc2a-d8dfba3a-468d7b6e,MONOCHROME2
297120,15132645,22796487,[R7611],21b2259f-4fe2fcd5-ae96904e-37327542-80d36be9,MONOCHROME2


In [ ]:
# Filter rows with MONOCHROME1
mono1_df = merged_df[merged_df['PhotometricInterpretation'] == 'MONOCHROME1']

# Filter rows with MONOCHROME2
mono2_df = merged_df[merged_df['PhotometricInterpretation'] == 'MONOCHROME2']

mono1_dicoms_unique = mono1_df['dicom'].unique().tolist()
mono2_dicoms_unique = mono2_df['dicom'].unique().tolist()

len(mono2_dicoms_unique)

123734

In [ ]:
# Step 1: Expand the dicom_ids column
expanded_df = subject_hadm_icd_dicom.explode('dicom_ids')

# Step 2: Merge with dicom_mono on the dicom column
merged_df = expanded_df.merge(dicom_mono[dicom_mono['PhotometricInterpretation'] == 'MONOCHROME2'],
                              left_on='dicom_ids', right_on='dicom', how='inner')

# Step 3: Select the desired columns
result = merged_df[['dicom', 'icd_codes']]

# Group by 'dicom' and aggregate 'icd_codes' as a single list
result = result.groupby('dicom')['icd_codes'].agg(lambda x: [code for codes in x for code in codes]).reset_index()

result

,dicom,icd_codes
0,00000218-9fb20d4e-86045713-8013e08b-0d5bebba,[J189]
1,0000d3be-591ae3b7-b03a7497-8319c02b-650bb4ab,[V0382]
2,0001bcde-bb754f26-6b32a474-90653a5d-3730aa89,"[4829, 49121, 49121, 49121, 49121, V1261, 4821..."
3,0002c540-fbe0f7cc-48725501-5ab948f7-8af53bbf,"[4821, 4820]"
4,0003494b-20c32dda-59f01833-783fbd4b-8d48e435,[J95851]
...,...,...
123729,fffd4d33-422d4d5b-7c09efc3-43832cf6-87c52a7f,"[J189, V0382]"
123730,fffea180-bc36450c-fb6d824a-4d089ba8-136b18c5,"[486, 486]"
123731,ffff5313-458549ff-8ef54533-add9597f-a0db64b7,"[4829, 79552, 49122, 486, V1201, 486, 79551, 7..."
123732,ffffc036-cbe84497-db8882f4-6bc3eada-fc5ca53f,"[5060, 5060, 49121, 49121, 49121, 49121, V1261..."
